# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
A. Pillepich  ->  A. Pillepich  |  ['A. Pillepich']
J. Shi  ->  J. Shi  |  ['J. Shi']
Arxiv has 78 new papers today
          6 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/6 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2409.03168


extracting tarball to tmp_2409.03168...

 done.
Retrieving document from  https://arxiv.org/e-print/2409.03522


extracting tarball to tmp_2409.03522...

...

 done.


list index out of range
Retrieving document from  https://arxiv.org/e-print/2409.03706


extracting tarball to tmp_2409.03706...

 done.


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/tmp/ipykernel_2646/1211882699.py:51: LatexWarning: 2409.03706 did not run properly
'PosixPath' object is not subscriptable
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.03522-b31b1b.svg)](https://arxiv.org/abs/2409.03522) | **Euclid preparation. Simulations and nonlinearities beyond $\Lambda$CDM. 1. Numerical methods and validation**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-09-06*|
|*Comments*| *20 pages, 7 figures, 1 appendix; submitted on behalf of the Euclid Collaboration*|
|**Abstract**|            To constrain models beyond $\Lambda$CDM, the development of the Euclid analysis pipeline requires simulations that capture the nonlinear phenomenology of such models. We present an overview of numerical methods and $N$-body simulation codes developed to study the nonlinear regime of structure formation in alternative dark energy and modified gravity theories. We review a variety of numerical techniques and approximations employed in cosmological $N$-body simulations to model the complex phenomenology of scenarios beyond $\Lambda$CDM. This includes discussions on solving nonlinear field equations, accounting for fifth forces, and implementing screening mechanisms. Furthermore, we conduct a code comparison exercise to assess the reliability and convergence of different simulation codes across a range of models. Our analysis demonstrates a high degree of agreement among the outputs of different simulation codes, providing confidence in current numerical methods for modelling cosmic structure formation beyond $\Lambda$CDM. We highlight recent advances made in simulating the nonlinear scales of structure formation, which are essential for leveraging the full scientific potential of the forthcoming observational data from the Euclid mission.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.03523-b31b1b.svg)](https://arxiv.org/abs/2409.03523) | **Euclid preparation. Simulations and nonlinearities beyond $\Lambda$CDM. 2. Results from non-standard simulations**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-09-06*|
|*Comments*| *22 pages, 7 figures*|
|**Abstract**|            The Euclid mission will measure cosmological parameters with unprecedented precision. To distinguish between cosmological models, it is essential to generate realistic mock observables from cosmological simulations that were run in both the standard $\Lambda$-cold-dark-matter ($\Lambda$CDM) paradigm and in many non-standard models beyond $\Lambda$CDM. We present the scientific results from a suite of cosmological N-body simulations using non-standard models including dynamical dark energy, k-essence, interacting dark energy, modified gravity, massive neutrinos, and primordial non-Gaussianities. We investigate how these models affect the large-scale-structure formation and evolution in addition to providing synthetic observables that can be used to test and constrain these models with Euclid data. We developed a custom pipeline based on the Rockstar halo finder and the nbodykit large-scale structure toolkit to analyse the particle output of non-standard simulations and generate mock observables such as halo and void catalogues, mass density fields, and power spectra in a consistent way. We compare these observables with those from the standard $\Lambda$CDM model and quantify the deviations. We find that non-standard cosmological models can leave significant imprints on the synthetic observables that we have generated. Our results demonstrate that non-standard cosmological N-body simulations provide valuable insights into the physics of dark energy and dark matter, which is essential to maximising the scientific return of Euclid.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.03524-b31b1b.svg)](https://arxiv.org/abs/2409.03524) | **Euclid preparation. Simulations and nonlinearities beyond $\Lambda$CDM. 4. Constraints on $f(R)$ models from the photometric primary probes**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-09-06*|
|*Comments*| *24 pages, 16 figures, submitted on behalf of the Euclid Collaboration*|
|**Abstract**|            We study the constraint on $f(R)$ gravity that can be obtained by photometric primary probes of the Euclid mission. Our focus is the dependence of the constraint on the theoretical modelling of the nonlinear matter power spectrum. In the Hu-Sawicki $f(R)$ gravity model, we consider four different predictions for the ratio between the power spectrum in $f(R)$ and that in $\Lambda$CDM: a fitting formula, the halo model reaction approach, ReACT and two emulators based on dark matter only $N$-body simulations, FORGE and e-Mantis. These predictions are added to the MontePython implementation to predict the angular power spectra for weak lensing (WL), photometric galaxy clustering and their cross-correlation. By running Markov Chain Monte Carlo, we compare constraints on parameters and investigate the bias of the recovered $f(R)$ parameter if the data are created by a different model. For the pessimistic setting of WL, one dimensional bias for the $f(R)$ parameter, $\log_{10}|f_{R0}|$, is found to be $0.5 \sigma$ when FORGE is used to create the synthetic data with $\log_{10}|f_{R0}| =-5.301$ and fitted by e-Mantis. The impact of baryonic physics on WL is studied by using a baryonification emulator BCemu. For the optimistic setting, the $f(R)$ parameter and two main baryon parameters are well constrained despite the degeneracies among these parameters. However, the difference in the nonlinear dark matter prediction can be compensated by the adjustment of baryon parameters, and the one-dimensional marginalised constraint on $\log_{10}|f_{R0}|$ is biased. This bias can be avoided in the pessimistic setting at the expense of weaker constraints. For the pessimistic setting, using the $\Lambda$CDM synthetic data for WL, we obtain the prior-independent upper limit of $\log_{10}|f_{R0}|< -5.6$. Finally, we implement a method to include theoretical errors to avoid the bias.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.03585-b31b1b.svg)](https://arxiv.org/abs/2409.03585) | **Extragalactic Stellar Tidal Streams: Observations meet Simulation**  |
|| J. Miro-Carretero, et al. -- incl., <mark>A. Pillepich</mark> |
|*Appeared on*| *2024-09-06*|
|*Comments*| *18 pages, 17 figures*|
|**Abstract**|            According to the well established hierarchical framework for galaxy evolution, galaxies grow through mergers with other galaxies and the LambdaCDM cosmological model predicts that the stellar halos of galaxies are rich in remnants from minor mergers. The Stellar Streams Legacy Survey has provided a first release of a catalogue with a statistically significant sample of stellar streams in the Local Universe that can be used to study minor mergers and test the cosmological models. The main objective is to compare the results of the observations of stellar tidal streams with the predictions of state-of-the-art cosmological simulations regarding the formation of stellar streams up to a redshift z < 0.02, according to the LambdaCDM model. We use the predictions of the cosmological simulations Copernicus Complexio, TNG50 of the IllustrisTNG project and Auriga to generate 225 mock-images of nearby halos at a distance of 70 Mpc, and search for stellar streams. We compare the obtained stream frequency and characteristics with those obtained from the Stellar Streams Legacy Survey. We find good agreement between the results of analysing real images from the Dark Energy Survey and mock-images from cosmological simulations. We obtained predictions for the detection rate of stellar streams to a surface brightness limit of 35 mag arcsec^-2. The cosmological simulations predict that for a surface brightness limit of 32 mag arcsec^-2 a frequency of almost 70% in the detection of streams around galaxies can be achieved.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.03168-b31b1b.svg)](https://arxiv.org/abs/2409.03168) | **The HI reservoir in central spiral galaxies and the implied star formation process**  |
|| J. Dou, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2024-09-06*|
|*Comments*| *18 pages, 7 figures; Accepted for publication in the ApJL; This is the fourth paper in the "From Haloes to Galaxies" series*|
|**Abstract**|            The cold interstellar medium (ISM) as the raw material for star formation is critical to understanding galaxy evolution. It is generally understood that galaxies stop making stars when, in one way or another, they run out of gas. However, here we provide evidence that central spiral galaxies remain rich in atomic gas even if their star formation rate and molecular gas fraction have dropped significantly compared to "normal" star-forming galaxies of the same mass. Since HI is sensitive to external processes, here we investigate central spiral galaxies using a combined sample from SDSS, ALFALFA, and xGASS surveys. After proper incompleteness corrections, we find that the key HI scaling relations for central spirals show significant but regular systematic dependence on stellar mass. At any given stellar mass, the HI gas mass fraction is about constant with changing specific star formation rate (sSFR), which suggests that HI reservoir is ubiquitous in central spirals with any star formation status down to M* ~ 10^9 Msun. Together with the tight correlation between the molecular gas mass fraction and sSFR for galaxies across a wide range of different properties, it suggests that the decline of SFR of all central spirals in the local universe is due to the halt of H2 supply, though there is plenty of HI gas around. These hence provide critical observations of the dramatically different behavior of the cold multi-phase ISM, and a key to understand the star formation process and quenching mechanism.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.03706-b31b1b.svg)](https://arxiv.org/abs/2409.03706) | **The Gaia Ultracool Dwarf Sample -- IV. GTC/OSIRIS optical spectra of Gaia late-M and L dwarfs**  |
|| W. J. Cooper, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2024-09-06*|
|*Comments*| *33 pages, 14 figures, Accepted by MNRAS*|
|**Abstract**|            As part of our comprehensive, ongoing characterisation of the low-mass end of the main sequence in the Solar neighbourhood, we used the OSIRIS instrument at the 10.4 m Gran Telescopio Canarias to acquire low- and mid-resolution (R$\approx$300 and R$\approx$2500) optical spectroscopy of 53 late-M and L ultracool dwarfs. Most of these objects are known but poorly investigated and lacking complete kinematics. We measured spectral indices, determined spectral types (six of which are new) and inferred effective temperature and surface gravity from BT-Settl synthetic spectra fits for all objects. We were able to measure radial velocities via line centre fitting and cross correlation for 46 objects, 29 of which lacked previous radial velocity measurements. Using these radial velocities in combination with the latest Gaia DR3 data, we also calculated Galactocentric space velocities. From their kinematics, we identified two candidates outside of the thin disc and four in young stellar kinematic groups. Two further ultracool dwarfs are apparently young field objects: 2MASSW J1246467$+$402715 (L4$\beta$), which has a potential, weak lithium absorption line, and G 196$-$3B (L3$\beta$), which was already known as young due to its well-studied primary companion.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error 'PosixPath' object is not subscriptable</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2409.03522/./Figures/Pk_comparison_LCDM.png', 'tmp_2409.03522/./Figures/CompCost_Summary.png', 'tmp_2409.03522/./Figures/Bk_comparison_fR.png']
copying  tmp_2409.03522/./Figures/Pk_comparison_LCDM.png to _build/html/
copying  tmp_2409.03522/./Figures/CompCost_Summary.png to _build/html/
copying  tmp_2409.03522/./Figures/Bk_comparison_fR.png to _build/html/
exported in  _build/html/2409.03522.md
    + _build/html/tmp_2409.03522/./Figures/Pk_comparison_LCDM.png
    + _build/html/tmp_2409.03522/./Figures/CompCost_Summary.png
    + _build/html/tmp_2409.03522/./Figures/Bk_comparison_fR.png
found figures ['tmp_2409.03523/./Figs/Pk_comparison_NewElephant.png', 'tmp_2409.03523/./Figs/Pk_comparison_Forge.png', 'tmp_2409.03523/./Figs/Pk_comparison_CiDER.png']
copying  tmp_2409.03523/./Figs/Pk_comparison_NewElephant.png to _build/html/
copying  tmp_2409.03523/./Figs/Pk_comparison_Forge.png to _build/html/
copying  tmp_2409.03523/./Figs/Pk_comparison_CiDER.png to _build/html/
e

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\Om}[1]{\Omega_\mathrm{#1}}$
$\newcommand{\Omm}{\Omega_\mathrm{m}}$
$\newcommand{\Omr}{\Omega_\mathrm{r}}$
$\newcommand{\Omk}{\Omega_\mathrm{k}}$
$\newcommand{\OmDE}{\Omega_\mathrm{DE}}$
$\newcommand{\OmL}{\Omega_\Lambda}$
$\newcommand{\de}[1]{\delta_\mathrm{#1}}$
$\newcommand{\rh}[1]{\rho_\mathrm{#1}}$
$\newcommand{\pysco}{\texttt{PySCo}\xspace}$
$\newcommand{\orcid}[1]{\orcidlink{#1}}$
$\newcommand{\arraystretch}{2.0}$</div>



<div id="title">

# $\Euclid$ preparation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.03522-b31b1b.svg)](https://arxiv.org/abs/2409.03522)<mark>Appeared on: 2024-09-06</mark> -  _20 pages, 7 figures, 1 appendix; submitted on behalf of the Euclid Collaboration_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** To constrain models beyond $\Lambda$ CDM, the development of the $\Euclid$ analysis pipeline requires simulations that capture the nonlinear phenomenology of such models.  We present an overview of numerical methods and $N$ -body simulation codes developed to study the nonlinear regime of structure formation in alternative dark energy and modified gravity theories. We review a variety of numerical techniques and approximations employed in cosmological $N$ -body simulations to model the complex phenomenology of scenarios beyond $\Lambda$ CDM. This includes discussions on solving nonlinear field equations, accounting for fifth forces, and implementing screening mechanisms. Furthermore, we conduct a code comparison exercise to assess the reliability and convergence of different simulation codes across a range of models. Our analysis demonstrates a high degree of agreement among the outputs of different simulation codes, providing confidence in current numerical methods for modelling cosmic structure formation beyond $\Lambda$ CDM. We highlight recent advances made in simulating the nonlinear scales of structure formation, which are essential for leveraging the full scientific potential of the forthcoming observational data from the $\Euclid$ mission.

</div>

<div id="div_fig1">

<img src="tmp_2409.03522/./Figures/Pk_comparison_LCDM.png" alt="Fig2" width="100%"/>

**Figure 2. -** Matter power spectra from simulations carried out with different codes (different line styles) for a $\Lambda$CDM cosmology. The *left panels* show the spectra at redshift $z=1$ whereas the *right panels* show the spectra at redshift $z=0.667$. The *bottom panels* show the relative difference with respect to the simulation carried out with \texttt{MG-Arepo}. (*fig:Pk_LCDM*)

</div>
<div id="div_fig2">

<img src="tmp_2409.03522/./Figures/CompCost_Summary.png" alt="Fig1" width="100%"/>

**Figure 1. -** Comparison of the computational costs of simulations run under different gravity models (labels on the $x$-axis) and with different codes (markers and colours as described in the legend). (*fig:CompCost*)

</div>
<div id="div_fig3">

<img src="tmp_2409.03522/./Figures/Bk_comparison_fR.png" alt="Fig3" width="100%"/>

**Figure 3. -** Amplification factor of the matter power spectra from simulations of two $f(R)$ scenarios, $\bar{f}_{,R} = 10^{-5}$(F5, *top row*) and $\bar{f}_{,R} = 10^{-6}$(F6, *bottom row*), relative to the reference $\Lambda$CDM cosmology. The amplification factor $B_P = P/P_{\Lambda\mathrm{CDM}}$ is measured for different codes (different line styles) at two different redshifts, $z=1$(*left panels*) and $z=0.667$(*right panels*). The *bottom panel* of each plot shows the relative agreement of the individual measurements, using \texttt{MG-Arepo} as a common reference. (*fig:Bk_fR*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.03522"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\lcdm}{\LambdaCDM\xspace}$
$\newcommand{\wcdm}{wCDM\xspace}$
$\newcommand{\om}{\Omega_\mathrm{m}}$
$\newcommand{\ob}{\Omega_\mathrm{b}}$
$\newcommand{\ol}{\Omega_\Lambda}$
$\newcommand{\ode}{\Omega_{\rm de}}$
$\newcommand{\odm}{\Omega_{\rm dm}}$
$\newcommand{\fR}{f_\mathrm{R}}$
$\newcommand{\fRz}{\overline{f_\mathrm{R0}}}$
$\newcommand{\horc}{H_0r_\mathrm{c}}$
$\newcommand{\de}{\rm de}$
$\newcommand{\neff}{N_{\rm eff}}$
$\newcommand{\kmsmpc}{  \kmsMpc}$
$\newcommand{\kpcoh}{  h^{-1}  \mathrm{kpc}}$
$\newcommand{\mpcoh}{  \hMpc}$
$\newcommand{\gpcoh}{  h^{-1}  \mathrm{Gpc}}$
$\newcommand{\gpcohcube}{  h^{-3}  \mathrm{Gpc}^3}$
$\newcommand{\homopc}{  h \mathrm{Mpc}^{-1}}$
$\newcommand{\msoh}{h^{-1} \ensuremath{{M}_{\odot}} }$
$\newcommand{\energydensity}{u}$
$\newcommand{\floor}{  \mathrm{floor}}$
$\newcommand{\euclid}{\textit{Euclid}\xspace}$
$\newcommand{\PGadgetTwo}{\texttt{p-GADGET2}\xspace}$
$\newcommand{\PGadgetThree}{\texttt{p-GADGET3}\xspace}$
$\newcommand{\ECOSMOG}{\texttt{ECOSMOG}\xspace}$
$\newcommand{\COLA}{\texttt{MG-COLA}\xspace}$
$\newcommand{\RAMSES}{\texttt{RAMSES}\xspace}$
$\newcommand{\FASTPM}{\texttt{FastPM}\xspace}$
$\newcommand{\AREPO}{\texttt{Arepo}\xspace}$
$\newcommand{\MGAREPO}{\texttt{MG-Arepo}\xspace}$
$\newcommand{\CGADGET}{\texttt{c-GADGET}\xspace}$
$\newcommand{\LGADGET}{\texttt{L-GADGET}\xspace}$
$\newcommand{\GADGETTWO}{\texttt{GADGET-2}\xspace}$
$\newcommand{\GADGET}{\texttt{GADGET}\xspace}$
$\newcommand{\MGGADGET}{\texttt{MG-GADGET}\xspace}$
$\newcommand{\GEVOLUTIONONETWO}{\texttt{gevolution-1.2}\xspace}$
$\newcommand{\GIZMO}{\texttt{GIZMO}\xspace}$
$\newcommand{\KEVOLUTION}{\texttt{k-evolution}\xspace}$
$\newcommand{\COMPLEMENTARY}{\textsc{Complementary}\xspace}$
$\newcommand{\DEMNUni}{\textsc{DEMNUni}\xspace}$
$\newcommand{\RAYGAL}{\textsc{Raygal}\xspace}$
$\newcommand{\ELEPHANT}{\textsc{Elephant}\xspace}$
$\newcommand{\ColaHiRes}{\textsc{COLA HiRes}\xspace}$
$\newcommand{\DUSTGRAIN}{\textsc{DUSTGRAIN}\xspace}$
$\newcommand{\DUSTGRAINPATHFINDER}{\textsc{DUSTGRAIN-PF}\xspace}$
$\newcommand{\CIDER}{\textsc{CiDER}\xspace}$
$\newcommand{\DAKAR}{\textsc{DAKAR}\xspace}$
$\newcommand{\DAKARTWO}{\textsc{DAKAR2}\xspace}$
$\newcommand{\DAKARONEANDTWO}{\textsc{DAKAR (1\&2)}\xspace}$
$\newcommand{\ClusteringDE}{\textsc{Clustering DE}\xspace}$
$\newcommand{\FORGE}{\textsc{FORGE}\xspace}$
$\newcommand{\BRIDGE}{\textsc{BRIDGE}\xspace}$
$\newcommand{\PNGUNITsim}{\textsc{PNG-UNIT}\xspace}$
$\newcommand{\UNITsim}{\textsc{UNIT}\xspace}$
$\newcommand{\rockstar}{\texttt{Rockstar}\xspace}$
$\newcommand{\nbodykit}{\texttt{nbodykit}\xspace}$
$\newcommand{\VIDE}{\texttt{VIDE}\xspace}$
$\newcommand{\ZOBOV}{\texttt{ZOBOV}\xspace}$
$\newcommand{\MPGRAFIC}{\texttt{MPGRAFIC}\xspace}$
$\newcommand{\NGENIC}{\texttt{N-GenIC}\xspace}$
$\newcommand{\TWOLPTIC}{\texttt{2LPTic}\xspace}$
$\newcommand{\CAMB}{\texttt{CAMB}\xspace}$
$\newcommand{\COMET}{\texttt{COMET}\xspace}$
$\newcommand{\COLOSSUS}{\texttt{COLOSSUS}\xspace}$
$\newcommand{\numpy}{\texttt{NumPy}\xspace}$
$\newcommand{\pandas}{\texttt{Pandas}\xspace}$
$\newcommand{\polars}{\texttt{Polars}\xspace}$
$\newcommand{\gam}[1]{\textcolor{darkgreen}{\bf Gary: #1}}$
$\newcommand{\MB}[1]{\textcolor{blue}{\bf MB: #1}}$
$\newcommand{\BF}[1]{\textcolor{olive}{\bf BF: #1}}$
$\newcommand{\PSC}[1]{\textcolor{purple}{\bf PSC: #1}}$
$\newcommand{\PSCtext}[1]{\textcolor{purple}{#1}}$
$\newcommand{\zs}[1]{\textcolor{brown}{#1}}$
$\newcommand{\AMCLB}[1]{\textcolor{orange}{\bf AMCLB: #1}}$
$\newcommand{\AMCLBtext}[1]{\textcolor{orange}{#1}}$
$\newcommand{\AK}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\GR}[1]{\textcolor{red}{#1}}$
$\newcommand$
$\newcommand{\HAW}[1]{\textcolor{red}{\bf Hans: #1}}$
$\newcommand{\orcid}[1]{\orcidlink{#1}}$
$\newcommand$</div>



<div id="title">

# $\Euclid$ preparation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.03523-b31b1b.svg)](https://arxiv.org/abs/2409.03523)<mark>Appeared on: 2024-09-06</mark> -  _22 pages, 7 figures_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The $\euclid$ mission will measure cosmological parameters with unprecedented precision. To distinguish between cosmological models, it is essential to generate realistic mock observables from cosmological simulations that were run in both the standard $\Lambda$ -cold-dark-matter ( $\lcdm$ ) paradigm and in many non-standard models beyond $\lcdm$ .  We present the scientific results from a suite of cosmological $_ N_$ -body simulations using non-standard models including dynamical dark energy, _k_ -essence, interacting dark energy, modified gravity, massive neutrinos, and primordial non-Gaussianities. We investigate how these models affect the large-scale-structure formation and evolution in addition to providing synthetic observables that can be used to test and constrain these models with $\euclid$ data.  We developed a custom pipeline based on the $\rockstar$ halo finder and the $\nbodykit$ large-scale structure toolkit to analyse the particle output of non-standard simulations and generate mock observables such as halo and void catalogues, mass density fields, and power spectra in a consistent way. We compare these observables with those from the standard $\lcdm$ model and quantify the deviations.  We find that non-standard cosmological models can leave significant imprints on the synthetic observables that we have generated. Our results demonstrate that non-standard cosmological $_ N_$ -body simulations provide valuable insights into the physics of dark energy and dark matter, which is essential to maximising the scientific return of $\euclid$ .

</div>

<div id="div_fig1">

<img src="tmp_2409.03523/./Figs/Pk_comparison_NewElephant.png" alt="Fig5" width="100%"/>

**Figure 5. -** Calculated matter and halo power spectra of the $\ELEPHANT$ simulation suite in the mass bin $10^{12.7}$\msoh$<M_{\rm halo}<10^{13.2}$\msoh$$. The solid lines represent the reference $\lcdm$ simulations, while the dashed lines the results of the nDGP simulations. _Top left:_ Real-space power spectra for dark matter. The dots above the solid lines highlight the locations where the power spectrum is estimated. _Top right:_ Real-space power spectra for haloes. _Bottom left:_ Monopole of the halo power spectrum in redshift space. _Bottom right:_ Quadrupole of the halo power spectrum in redshift space. (*fig:ElephantPkComparison*)

</div>
<div id="div_fig2">

<img src="tmp_2409.03523/./Figs/Pk_comparison_Forge.png" alt="Fig6" width="100%"/>

**Figure 6. -** Calculated matter and halo power spectra from the $\FORGE$ simulation suite in the mass bin $10^{12.7}$\msoh$<M_{\rm halo}<10^{13.2}$\msoh$$. The solid lines represent the reference $\lcdm$ simulations, while the dashed lines are the results of the nDGP simulations. _Top left:_ Real-space power spectra for dark matter. The dots above the solid lines highlight the locations where the power spectrum is estimated. _Top right:_ Real-space power spectra for haloes. _Bottom left:_ Monopole of the halo power spectrum in redshift space. _Bottom right:_ Quadrupole of the halo power spectrum in redshift space. (*fig:ForgePkComparison*)

</div>
<div id="div_fig3">

<img src="tmp_2409.03523/./Figs/Pk_comparison_CiDER.png" alt="Fig7" width="100%"/>

**Figure 7. -** Calculated matter and halo power spectra from the $\CIDER$ simulation suite in the mass bin $10^{12.7}$\msoh$<M_{\rm halo}<10^{13.2}$\msoh$$. The solid lines represent the reference $\lcdm$ simulations, while the dashed lines are the results of the IDE simulations. _Top left:_ Real-space power spectra for dark matter. The dots above the solid lines highlight the locations where the power spectrum is estimated. _Top right:_ Real-space power spectra for haloes. _Bottom left:_ Monopole of the halo power spectrum in redshift space. _Bottom right:_ Quadrupole of the halo power spectrum in redshift space. (*fig:CiderPkComparison*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.03523"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\bfpsi}{\bm\psi}$
$\newcommand{\bfell}{\bm\ell}$
$\newcommand{\bfn}{\bm n}$
$\newcommand{\bfth}{\bm\theta}$
$\newcommand{\bfz}{\bm z}$
$\newcommand{\bfs}{\bm s}$
$\newcommand{\bfv}{\bm v}$
$\newcommand{\bfx}{\bm x}$
$\newcommand{\bfk}{\bm k}$
$\newcommand{\bfp}{\bm p}$
$\newcommand{\bfq}{\bm q}$
$\newcommand{\bfr}{\bm r}$
$\newcommand{\bfu}{\bm u}$
$\newcommand{\bfw}{\bm w}$
$\newcommand{\bfy}{\bm y}$
$\newcommand{\bfPhi}{\bm\Phi}$
$\newcommand{\bfM}{\bm M}$
$\newcommand{\bfF}{\bm F}$
$\newcommand{\fnl}{f_{\rm NL}}$
$\newcommand{\gnl}{g_{\rm NL}}$
$\newcommand{\tdu}{\widetilde{u}}$
$\newcommand{\tdPhi}{\widetilde{\Phi}}$
$\newcommand{\tdG}{\widetilde{G}}$
$\newcommand{\tdP}{\widetilde{P}}$
$\newcommand{\tdR}{\widetilde{R}}$
$\newcommand{\threextwo}{\text{3\times2pt}}$
$\newcommand{\GCsp}{\text{GC}\ensuremath{_\mathrm{sp}}}$
$\newcommand{\GCph}{\text{GC}\ensuremath{_\mathrm{ph}}}$
$\newcommand{\XCph}{\text{XC}\ensuremath{_\mathrm{ph}}}$
$\newcommand{\Omegam}{\ensuremath{\Omega_{\mathrm{m}}}}$
$\newcommand{\Omegab}{\ensuremath{\Omega_{\mathrm{b}}}}$
$\newcommand{\Omegac}{\ensuremath{\Omega_{\mathrm{c}}}}$
$\newcommand{\Omeganu}{\ensuremath{\Omega_{\nu}}}$
$\newcommand{\OmegaDE}{\ensuremath{\Omega_{\mathrm{DE}}}}$
$\newcommand{\ns}{n_{\mathrm{s}}}$
$\newcommand{\lcdm}{\ensuremath{\Lambda\mathrm{CDM}}}$
$\newcommand{\logfr}{\ensuremath{\logten|f_{R0}|}}$
$\newcommand{\fr}{\ensuremath{|f_{R0}|}}$
$\newcommand{\sg}{\ensuremath{\sigma_{8}}}$
$\newcommand{\de}{\mathrm{d}}$
$\newcommand{\arepo}{\texttt{Arepo}}$
$\newcommand{\mgarepo}{\texttt{MG-AREPO}}$
$\newcommand{\ecosmog}{\texttt{ECOSMOG}}$
$\newcommand{\forge}{\texttt{FORGE}}$
$\newcommand{\react}{\texttt{ReACT}}$
$\newcommand{\emantis}{\texttt{e-Mantis}}$
$\newcommand{\ramses}{\texttt{RAMSES}}$
$\newcommand{\halofit}{\texttt{halofit}}$
$\newcommand{\montepython}{\texttt{MontePython}}$
$\newcommand{\bcemu}{\texttt{BCemu}}$
$\newcommand{\eetwo}{\texttt{EuclidEmulator2}}$
$\newcommand{\bacco}{\texttt{Bacco}}$
$\newcommand{\hmcode}{\texttt{HMCode2020}}$
$\newcommand{\fofrfitting}{\texttt{fitting}}$
$\newcommand{\dustgrain}{\texttt{DUSTGRAIN}}$
$\newcommand{\elephant}{\texttt{ELEPHANT}}$
$\newcommand{\cosmopower}{\texttt{Cosmopower}}$
$\newcommand{\mggadget}{\texttt{MG-Gadget}}$
$\newcommand{\orcid}[1]$</div>



<div id="title">

# $\Euclid$ preparation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.03524-b31b1b.svg)](https://arxiv.org/abs/2409.03524)<mark>Appeared on: 2024-09-06</mark> -  _24 pages, 16 figures, submitted on behalf of the Euclid Collaboration_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** We study the constraint on $f(R)$ gravity that can be obtained by photometric primary probes of the $\Euclid$ mission. Our focus is the dependence of the constraint on the theoretical modelling of the nonlinear matter power spectrum. In the Hu--Sawicki $f(R)$ gravity model, we consider four different predictions for the ratio between the power spectrum in $f(R)$ and that in $\lcdm$ : a fitting formula, the halo model reaction approach, $\react$ and two emulators based on dark matter only $_ N_$ -body simulations, $\forge$ and $\emantis$ . These predictions are added to the $\montepython$ implementation to predict the angular power spectra for weak lensing (WL), photometric galaxy clustering and their cross-correlation. By running Markov Chain Monte Carlo, we compare constraints on parameters and investigate the bias of the recovered $f(R)$ parameter if the data are created by a different model. For the pessimistic setting of WL, one dimensional bias for the $f(R)$ parameter, $\logfr$ , is found to be $0.5 \sigma$ when $\forge$ is used to create the synthetic data with $\logfr =-5.301$ and fitted by $\emantis$ . The impact of baryonic physics on WL is studied by using a baryonification emulator $\bcemu$ . For the optimistic setting, the $f(R)$ parameter and two main baryon parameters are well constrained despite the degeneracies among these parameters. However, the difference in the nonlinear dark matter prediction can be compensated by the adjustment of baryon parameters, and the one-dimensional marginalised constraint on $\logfr$ is biased. This bias can be avoided in the pessimistic setting at the expense of weaker constraints. For the pessimistic setting, using the $\lcdm$ synthetic data for WL, we obtain the prior-independent upper limit of $\logfr < -5.6$ . Finally, we implement a method to include theoretical errors to avoid the bias due to inaccuracies in the nonlinear matter power spectrum prediction.

</div>

<div id="div_fig1">

<img src="tmp_2409.03524/././plots/WL_optbar1_prior.png" alt="Fig13.1" width="50%"/><img src="tmp_2409.03524/././plots/WL_optbar2_prior.png" alt="Fig13.2" width="50%"/>

**Figure 13. -** Constraints on parameters from the WL optimistic setting with baryons. We show the case where $\emantis$  is used to create both the data and fitting and the case where $\forge$  is used to create the data and it is fitted by $\emantis$ . We also include the case where we impose a Gaussian prior on $\logten M_{\rm c}$ with the width of 0.2 in the case of $\forge$  fiducial data.
  _Left panel_: Constraints on baryon parameters. _Right panel_: Constraints on cosmological parameters. (*fig:optbar*)

</div>
<div id="div_fig2">

<img src="tmp_2409.03524/././plots/baryon.png" alt="Fig12" width="100%"/>

**Figure 12. -** The effect of $\logfr$, two baryonic parameters $\logten M_{\rm c}$ and $\theta_{\rm ej}$, and $h$ on the WL angular power spectrum in bin 10. The dotted lines show predictions of $\emantis$  with varying these parameters individually. The plot shows the ratio to the $\lcdm$  power spectrum. The plot includes the prediction of $\forge$  and $\emantis$  with the fiducial parameters and the $\emantis$  prediction using the best-fit values
  ($\logfr=-5.765, 100 h= 65.736, \logten(M_{\rm c})=12.999$)
with the data created by $\forge$ . We can see that the $\emantis$  prediction can fit the $\forge$  by adjusting these parameters. We note that the cosmological parameters for the $\lcdm$  power spectrum $C_{\ell}^{$\lcdm$}$ is fixed to be the fiducial ones when we vary $100  h$.  (*fig:baryoneffect*)

</div>
<div id="div_fig3">

<img src="tmp_2409.03524/././plots/WL_pes_emantis.png" alt="Fig4" width="100%"/>

**Figure 4. -** Bias due to different nonlinear modelling. The synthetic data are created by $\emantis$  and fitted by two different models in WL pessimistic case without baryons.  (*fig:biasemantis*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.03524"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\apc}[1]{ \textcolor{purple}{\textbf{APC:} #1}}$
$\newcommand{\apcedit}[1]{\textbf{#1}}$</div>



<div id="title">

# Extragalactic Stellar Tidal Streams: Observations meet Simulation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.03585-b31b1b.svg)](https://arxiv.org/abs/2409.03585)<mark>Appeared on: 2024-09-06</mark> -  _18 pages, 17 figures_

</div>
<div id="authors">

J. Miro-Carretero, et al. -- incl., <mark>A. Pillepich</mark>

</div>
<div id="abstract">

**Abstract:** According to the well established hierarchical framework for galaxy evolution, galaxies grow through mergers with other galaxies and the $\Lambda$ CDM cosmological model predicts that the stellar halos of galaxies are rich in remnants from minor mergers. The _Stellar Streams Legacy Survey_ has provided a first release of a catalogue with a statistically significant sample of stellar streams in the Local Universe that can be used to study minor mergers and test the cosmological models. The main objective is to compare the results of the observations of stellar tidal streams with the predictions of state-of-the-art cosmological simulations regarding the formation of stellar streams up to a redshift z < 0.02, according to the $\Lambda$ CDM model. We use the predictions of the cosmological simulations _Copernicus Complexio, TNG50 of the IllustrisTNG project and Auriga_ to generate 225 mock-images of nearby halos at a distance of 70 Mpc, and search for stellar streams. We compare the obtained stream frequency and characteristics with those obtained from the _Stellar Streams Legacy Survey_ . We find  good agreement between the results of analysing real images from the _Dark Energy Survey_ and mock-images from cosmological simulations. We obtained predictions for the detection rate of stellar streams to a surface brightness limit of 35 mag arcsec $^{-2}$ The cosmological simulations predict that for a surface brightness limit of 32 mag arcsec $^{-2}$ a frequency of almost 70 \% in the detection of streams around galaxies can be achieved.

</div>

<div id="div_fig1">

<img src="tmp_2409.03585/./Histogram_Mstar_DES-COCO-TNG50-Auriga_240816-2.jpg" alt="Fig2" width="100%"/>

**Figure 2. -** Histogram of
  $\log_{10} {M_\star}$
  for the observed (grey) and simulated galaxy samples (blue, red and green for COCO, IllustrisTNG50 and Auriga, respectively).
  The _black vertical line_ shows the average of the DES sample ($\log_{10} (M_{\star} / \mathrm{M_{\odot}} = 10.00$) and the _dark grey vertical lines_ the
  $\pm$ 1$\sigma$ range around that average, [$9.62, 10.38 $] . The _light grey vertical line_ shows the average of the DES sample $+2\sigma = 10.76$.
  The maximum value of each histogram has been re-scaled to 1 for this comparison (*fig-stellarmassCOCODES*)

</div>
<div id="div_fig2">

<img src="tmp_2409.03585/./Histogram_Stellar-Mass_DES_CoCo_sample.jpg" alt="Fig5" width="100%"/>

**Figure 5. -** Distribution of host stellar mass in the DES sample and in the the COCO simulations within the selected stellar mass range. (*fig-histogramstelllarmass*)

</div>
<div id="div_fig3">

<img src="tmp_2409.03585/./AURIGA_TNG50_COCO_DES_Stream-Detecttability_240816.jpg" alt="Fig6" width="100%"/>

**Figure 6. -** Detection rate curves for the COCO, IllustrisTNG50 and Auriga cosmological simulations and the detection rate observed for the DES sample (_black square_ indicates the dispersion in surface brightness limit of the DES sample and the dispersion in the detection rate according to a binomial distribution, see \citet{miro-carretero2024}). The percentage of the streams detected by visual inspection is plotted versus the surface brightness limit at which such detection was possible. The bars indicate the Poisson confidence interval. (*fig-comparisondesonlyetectioncurve*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.03585"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

175  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
